close data for backtesting

In [7]:
from sqlalchemy import create_engine
from binance.client import Client
import pandas as pd
import pymysql

api_key = 'ihvXWbHvvDrYuroIJOS9ANW6X7tv5OQSYibLLloKyIGui0w93Q5yUJOBZclnNL5D'
api_secret = 'uzmKBJ3lmVS4TsTl2AOHVJOYTFfLM5qrg00vAqUyTYbSjiy7BDPlTpaq2Oy7qdo2'
client = Client(api_key, api_secret)

db_data = 'mysql+mysqldb://' + 'admin' + ':' + '0323A8E3DB' + '@' + 'coin-database-1-instance-1.c9hqcydvu19f.us-east-1.rds.amazonaws.com' + ':3306/' + 'crypto_db' + '?charset=utf8mb4'
engine = create_engine(db_data)


In [2]:
def getminutedata(symbol, start, end):
    frame = pd.DataFrame(client.get_historical_klines(symbol, Client.KLINE_INTERVAL_15MINUTE, start, end))
    frame = frame.iloc[:,:6]                                                    
    frame.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
    frame = frame.set_index('date')
    frame.index = pd.to_datetime(frame.index, unit= 'ms')
    frame = frame.astype(float)
    return frame.reset_index()

In [3]:
cryptoString = 'SELECT * FROM crypto'
crypto_table = pd.read_sql_query(cryptoString, engine)
full_id_list = crypto_table.id.astype(int).tolist()


In [13]:
sql_create_crypto_price_table = """CREATE TABLE IF NOT EXISTS crypto_price (
                    crypto_id int(7),
                    date datetime NOT NULL,
                    open decimal(30, 20) NOT NULL,
                    high decimal(30, 20) NOT NULL,
                    low decimal(30, 20) NOT NULL,
                    close decimal(30, 20) NOT NULL,
                    volume integer(20),
                    PRIMARY KEY(crypto_id, date),
                    FOREIGN KEY (crypto_id) REFERENCES crypto (id)
                );"""
conn = pymysql.connect(host='coin-database-1-instance-1.c9hqcydvu19f.us-east-1.rds.amazonaws.com', user='admin', password='0323A8E3DB', database='crypto_db')
c = conn.cursor()
c.execute(sql_create_crypto_price_table)

0

In [21]:
start_string = "2022-03-11 12:00:00"
end_string = "2022-03-11 13:45:00"

for id in full_id_list:
    df = getminutedata(crypto_table.loc[crypto_table['id'] == id]['ticker'].values[0], start_string, end_string)
    df['crypto_id'] = id
    df.to_sql("crypto_price", engine, if_exists="append", index=False)


In [ ]:
value = getminutedata(crypto_table.loc[crypto_table['id'] == crypto_id]['ticker'].values[0], 
                                                str(date_request.loc[crypto_id]['date_request'] - datetime.timedelta(hours=2)) ,
                                                str(date_request.loc[crypto_id]['date_request']) )
close = value[['date','tlose']]                             
close.index = pd.to_datetime(close['date'])
close = close.drop(columns=['date'])
close = close.rename(columns={"Close": f"{crypto_id}"})

In [16]:
date_lower_str = str(date_request.loc[crypto_id]['date_request'] - datetime.timedelta(hours=2))
date_upper_str = str(date_request.loc[crypto_id]['date_request'] - datetime.timedelta(hours=2))

new_close_string = f"SELECT date, close FROM crypto_price WHERE 'date' > '{date_lower_str}' AND 'date' <= '{date_upper_str}' ORDER BY date"
pd.read_sql_query(new_close_string, engine)

NameError: name 'date_request' is not defined

In [20]:
date_lower_str = "2022-03-11 16:00:00"
date_upper_str = "2022-03-12 17:00:00"
id = 1
new_close_string = f"SELECT date, close FROM crypto_price WHERE crypto_id = {id} AND date > '{date_lower_str}' AND date <= '{date_upper_str}' ORDER BY date"
pd.read_sql_query(new_close_string, engine)

,date,close
0,2022-03-11 16:15:00,1.314
1,2022-03-11 16:30:00,1.315
2,2022-03-11 16:45:00,1.318
3,2022-03-11 17:00:00,1.312
4,2022-03-11 17:15:00,1.314
...,...,...
95,2022-03-12 16:00:00,1.359
96,2022-03-12 16:15:00,1.370
97,2022-03-12 16:30:00,1.368
98,2022-03-12 16:45:00,1.366
